In [8]:
##  open Source models - Llama3 , Gemma2 ,mistral--Groq
import os
from dotenv  import load_dotenv

load_dotenv()
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

Create a coversational chat bot with history

In [9]:
from langchain_groq import ChatGroq     
model = ChatGroq( model_name="gemma2-9b-it",groq_api_key=groq_api_key)
## update api key 

In [10]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi i am das trying to learn gen ai")])

AIMessage(content="Hello Das! \n\nIt's great that you're interested in learning about generative AI! It's a fascinating and rapidly evolving field. \n\nTo help me guide you, could you tell me a little more about what you'd like to learn? For example:\n\n* **What's your current level of understanding of AI?** (Beginner, some knowledge, etc.)\n* **What specifically interests you about generative AI?** (Text generation, image creation, music composition, etc.)\n* **What are your goals for learning about generative AI?** (Personal curiosity, professional development, etc.)\n\nOnce I have a better understanding of your background and interests, I can provide you with more relevant and helpful information.\n\nIn the meantime, here are some general resources that you might find useful:\n\n* **OpenAI's blog:** https://openai.com/blog/\n* **Google AI blog:** https://ai.googleblog.com/\n* **Stanford's CS224N: Natural Language Processing with Deep Learning:** https://web.stanford.edu/class/cs224n

In [11]:
from langchain_core.messages import AIMessage
model.invoke([HumanMessage(content="Hi my name is Das , i am tring to become a gen ai deeveloper"),AIMessage(content="Hi das, how can I help you today?"),
              HumanMessage(content="What i am doing what are my interests")])

AIMessage(content="It's great that you're interested in becoming a Gen AI developer! That's a really exciting field right now.  \n\nSince you're just starting out,  it's helpful to think about what specifically interests you about Gen AI.  Here are some areas you could explore:\n\n**Core Technologies:**\n\n* **Deep Learning:** This is the foundation of most Gen AI models. Understanding concepts like neural networks, transformers, and different training techniques is essential.\n* **Natural Language Processing (NLP):** If you're interested in text generation, translation, or understanding human language, NLP is key.\n* **Computer Vision:** For generating images, videos, or understanding visual content, computer vision skills are important.\n\n**Applications:**\n\n* **Text Generation:**  Chatbots, story writing, code generation, summarization, dialogue systems.\n* **Image Generation:** Creating new images from text descriptions, editing existing images, generating art.\n* **Multimodal AI

###  message history 
we can use a message history to keep track of the conversation and make it statepful

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

  

In [13]:
config = {"configurable":{"session_id": "chat1"}}

In [14]:
response =with_message_history.invoke(
    [HumanMessage(content="Hi my name is Das , i am tring to become a gen ai deeveloper")],
    config=config
)

In [15]:
response.content

"That's fantastic, Das!  Becoming a Gen AI developer is an exciting and challenging path. \n\nTo help me give you the best advice, could you tell me a little more about yourself and your goals?\n\nFor example:\n\n* **What's your current background?** (e.g., are you a programmer, a data scientist, or just starting out?)\n* **What specific areas of Gen AI are you interested in?** (e.g., text generation, image generation, code generation, etc.)\n* **What are your learning resources and preferences?** (e.g., online courses, books, tutorials, mentorship)\n\nThe more information you share, the better I can tailor my suggestions to your needs. \n\nI'm here to support you on your journey! Let's get started. 🎉\n"

In [16]:
response =with_message_history.invoke(
    [HumanMessage(content="What is my name ")],
    config=config
)

In [17]:
response.content

'Your name is Das. 😊  I remember that from our first conversation!  \n\nHow can I help you further today?\n'

In [18]:
## change in config 
config1 = {"configurable":{"session_id": "chat2"}}
response =with_message_history.invoke(
    [HumanMessage(content="What is my name ")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know!\n"

In [26]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens = 170,
    strategy = "last",
    token_counter = model,
    include_system =True,
    allow_partial  = False,
    start_on = "human"
) 

In [27]:
messages = [
    SystemMessage(content="You are a helpful AI assistant that answers questions precisely and concisely."),
    
    HumanMessage(content="Hey, can you explain what a virtual environment is in Python?"),
    AIMessage(content="Sure! A virtual environment in Python is a self-contained directory that contains a Python installation for a particular version, plus a number of additional packages. It helps isolate dependencies for different projects."),

    HumanMessage(content="Cool. How do I create one?"),
    AIMessage(content="You can create one using the command: `python -m venv myenv`. Replace `myenv` with the name you prefer."),

    HumanMessage(content="What about activating it on Windows?"),
    AIMessage(content="On Windows, run: `myenv\\Scripts\\activate`. On macOS/Linux, it's `source myenv/bin/activate`."),

    HumanMessage(content="Nice. Can you list the commands to install FastAPI and MongoDB-related packages too?"),
    AIMessage(content="Sure! You can install them using: `pip install fastapi uvicorn pymongo motor`."),
]
trimmer.invoke(messages)

[SystemMessage(content='You are a helpful AI assistant that answers questions precisely and concisely.'),
 HumanMessage(content='Cool. How do I create one?'),
 AIMessage(content='You can create one using the command: `python -m venv myenv`. Replace `myenv` with the name you prefer.'),
 HumanMessage(content='What about activating it on Windows?'),
 AIMessage(content="On Windows, run: `myenv\\Scripts\\activate`. On macOS/Linux, it's `source myenv/bin/activate`."),
 HumanMessage(content='Nice. Can you list the commands to install FastAPI and MongoDB-related packages too?'),
 AIMessage(content='Sure! You can install them using: `pip install fastapi uvicorn pymongo motor`.')]

In [34]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content="You are a helpful AI assistant. Based on the conversation history, try to understand and infer the user's field of interest."
    ),
    ("user", "{messages}"),  # This will be replaced with the trimmed messages list
    ("user", "What is my field of interest?")
])
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

# Proper dictionary syntax for invoke
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What is my field of interest?")],
    "language": "English"
})

print(response)

content="Based on our conversation, your field of interest appears to be **backend software development**. \n\nYou're asking about creating a virtual environment, activating it, and installing packages like FastAPI and MongoDB, which are commonly used in building web applications.  \n\n" response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 233, 'total_tokens': 288, 'completion_time': 0.1, 'prompt_time': 0.009166909, 'queue_time': 0.23533897, 'total_time': 0.109166909}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-b667d7ae-a7ec-4943-ae74-eec15d355436-0' usage_metadata={'input_tokens': 233, 'output_tokens': 55, 'total_tokens': 288}


In [36]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What is question i asked you in python")],
    "language": "English"
})
response

AIMessage(content="Based on our conversation, your field of interest seems to be **Python development**, particularly focusing on building web applications using **FastAPI** and potentially working with **MongoDB** as a database. \n\n\nYou're asking about setting up a Python virtual environment, activating it, and installing specific packages related to these technologies. \n\n", response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 234, 'total_tokens': 303, 'completion_time': 0.125454545, 'prompt_time': 0.009277331, 'queue_time': 0.237394157, 'total_time': 0.134731876}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c24dac3c-e236-4478-ae8e-137c19c3251b-0', usage_metadata={'input_tokens': 234, 'output_tokens': 69, 'total_tokens': 303})

In [37]:
## wrap it in message history again
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable":{"session_id":"chat5"}}